# Colab에서 돌려보는 Faster R-CNN

해당 내용은 PULKIT SHARMA의 [A Practical Implementation of the Faster R-CNN Algorithm for Object Detection](https://www.analyticsvidhya.com/blog/2018/11/implementation-faster-r-cnn-python-object-detection/)을 번역한 내용입니다.

## Introduction

Object detection 알고리즘인 'Faster R-CNN'을 직접 실행해보자. 실험을 통해 Faster R-CNN이 얼마나 좋은 성능을 보여주는지 확인 할 수 있다.

개념을  이해하기위해 [' A Step-by-Step Introduction to the Basic Object Detection Algorithms (Part 1)'](https://www.analyticsvidhya.com/blog/2018/10/a-step-by-step-introduction-to-the-basic-object-detection-algorithms-part-1/)를 읽어보는 걸 추천한다.

## Table of Contents
1. A Brief Overview of the Different R-CNN Algorithms for Object Detection
2. Understanding the Problem Statement
3. Setting up the System
4. Data Exploration
5. Implementing Faster R-CNN

## A Brief Overview of the Different R-CNN Algorithms for Object Detection

Faster R-CNN의 Object detection 과정은 다음과 같다.

1. Input 이미지를 받아 Conv layer를 통과시켜 feature map을 얻는다.
2. 이 feature map에 Region Proposal Network(RPN)을 적용해서 object proposals를 얻는다.
3. 모든 proposals에 ROI pooling layer을 적용해서 동일한 크기(same size)로 만든다.
4. 해당 proposals를 Fully connected layer를 통과시켜 classificaion과 bounding box 값을 얻는다.


<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*e6dx5qzUKWwasIVGSuCyDA.png" width="400">
    <br>
    <font color=gray><i>Faster R-CNN</i><font>
</div>
<br>

아래 표는 다른 알고리즘을 요약한 내용이다.

<br>
<div align="center">
    <img src="https://user-images.githubusercontent.com/30274701/53162008-77de2500-360e-11e9-88bc-de5c9e301f99.png" width="600">
    <br>
    <font color=gray><i>Faster R-CNN</i><font>
</div>
<br>

## Understanding the Problem Statement

여기서는 'Blood Cell Detection' 문제를 해결해보자. 주어진 현미경 이미지에서  적혈구, 백혈구 및 혈소판 밀도(the Red Blood Cells (RBCs), White Blood Cells (WBCs), Platelets)을 찾는(detection) 문제이다. 최종 예상 결과는 아래 그림과 같다.

<br>
<div align="center">
    <img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/10/example.jpg" width="400">
    <br>
    <font color=gray><i>Faster R-CNN</i><font>
</div>
<br>

이 데이터를 선택하는 이유는 혈류의 적혈구, 백혈구 및 혈소판 밀도가 면역계와 헤모글로빈에 대한 많은 정보를 제공하기 때문이다. 이 데이터를 활용하면 잠재적으로 사람이 건강한지 아닌지를 확인하는 데 도움이 될 수 있으며, 혈액에 불일치가 발견되면 이를 신속하게 발견하여 진단 할 수 있다.

사람이 직접 현미경을 보며 진단하는 과정은 정확도가 낮고 고된 작업일 수 있다. 따라서 Deep learning Model이 중요한 역활을 할 수 있을 것이다. 

full blood cell detection dataset은 [여기](https://github.com/Shenggan/BCCD_Dataset)에서 받을 수 있다.

 해당 데이터셋에서 몇 가지를 수정할 것이며, 모델을 만드는 전체 과정은 keras와 tensorflow를 이용해서 만들어 볼 것이다.
 1. bounding boxes .xml 파일을 .csv format으로 변경
 2. 전체 데이터셋을 랜덤하게 train과 test 셋으로 나눔.

## Setting up the System

사용할 라이브러리(library)는 다음과 같다. 해당 라이브러리를 txt 파일로 만들어 쉽게 설치할 수 있다.

* pandas
* matplotlib
* tensorflow
* keras – 2.0.3
* numpy
* opencv-python
* sklearn
* h5py

다운로드 링크 : https://drive.google.com/file/d/1R4O0stMW9Wjksg-o7c54svntDiyask1B/view

In [ ]:
!pip install -r C:/Users/home/Downloads/requirement.txt

## Data Exploration

항상 데이터를 먼저 살펴보는 것이 중요하다. 전체적인 부분을 살펴보면서 어떻게 작업할 지 생각해보고 숨겨진 패턴을 찾을 수도 있다. 여기서는 데이터셋을 3가지 파일로 구성해서 학습을 진행 할 것이다.

1. train_image
2. test_image
3. train.csv

다운받은 데이터셋으로 csv파일은 새로 만들어야 한다. 저자가 사용한 코드는 다음과 같다. github에서 원본 레포를 clone한 후에, 로컬에서 csv 파일을 만들어 업로드하는 방법을 추천한다.

In [ ]:
%pylab inline

import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile

annotations = glob('C:/Users/home/source/BCCD_dataset/BCCD/Annotations/*.xml') #using your download path

df = []
cnt = 0
for file in annotations:
    prev_filename = file.split('/')[-1].split('.')[0] + '.jpg'
    
    #print(prev_filename)
    filename = str(cnt) + '.jpg'
    row = []
    parsedXML = ET.parse(file)
    for node in parsedXML.getroot().iter('object'):
        blood_cells = node.find('name').text
        xmin = int(node.find('bndbox/xmin').text)
        xmax = int(node.find('bndbox/xmax').text)
        ymin = int(node.find('bndbox/ymin').text)
        ymax = int(node.find('bndbox/ymax').text)

        row = [prev_filename[12:], filename, blood_cells, xmin, xmax, ymin, ymax]
        df.append(row)
    cnt += 1
    
data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])

data[['prev_filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('blood_cell_detection.csv', index=False)


padas 라이브러리를 사용해서 csv 파일을 살펴보자. 6개의 column으로 이루어져 있으며, 내용은 다음과 같다.

* image_names: contains the name of the image
* cell_type: denotes the type of the cell
* xmin: x-coordinate of the bottom left part of the image
* xmax: x-coordinate of the top right part of the image
* ymin: y-coordinate of the bottom left part of the image
* ymax: y-coordinate of the top right part of the image

In [ ]:
# importing required libraries
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
from matplotlib import patches

In [ ]:
# read the csv file using read_csv function of pandas
train = pd.read_csv('./keras-frcnn/blood_cell_detection.csv')
train.head()

이미지 파일은 해당 github를 클론해서 가져오면 편리하다.
https://github.com/Shenggan/BCCD_Dataset.git

불러온 이미지도 확인해보자.

In [ ]:
!git clone https://github.com/Shenggan/BCCD_Dataset.git

In [ ]:
# reading single image using imread function of matplotlib
image = plt.imread('c:/Users/home/source/BCCD_Dataset/BCCD/JPEGImages/BloodImage_00000.jpg')
plt.imshow(image)

이미지에서 파란색 부분이 백혈구(WBCs)이며, 살짝 붉은 부분이 적혈구(RBC)이다.

총 이미지의 숫자를 확인해보자. 위 코드를 사용하면 전체 이미지 364장을 모두 csv 파일로 저장하게 된다. 

테스트 셋을 만들기 위해서 254장 뒷부분을 테스트셋으로 구성하자. data.split을 사용하면 더 좋을 것이다.

In [ ]:
# Number of unique training images
train['prev_filename'].nunique()

In [ ]:
# Number of classes
train['cell_type'].value_counts()

In [ ]:
test_df = pd.read_csv('./keras-frcnn/blood_cell_detection.csv')
test_df = test_df[4738:] # 10 images

위 value_count를 보면, 3가지 class로 되어있는 것을 확인할 수 있다. 그럼 이미지에 bounding box를 더한 그림을 살펴보도록 하자.

In [ ]:
fig = plt.figure()

#add axes to the image
ax = fig.add_axes([0,0,1,1])

# read and plot the image
image = plt.imread('c:/Users/home/source/BCCD_Dataset/BCCD/JPEGImages/BloodImage_00000.jpg')
plt.imshow(image)

# iterating over the image for different objects
for _,row in train[train.prev_filename == "BloodImage_00000.jpg"].iterrows():
    xmin = row.xmin
    xmax = row.xmax
    ymin = row.ymin
    ymax = row.ymax
    
    width = xmax - xmin
    height = ymax - ymin
    
    # assign different color to different classes of objects
    if row.cell_type == 'RBC':
        edgecolor = 'r'
        ax.annotate('RBC', xy=(xmax-40,ymin+20))
    elif row.cell_type == 'WBC':
        edgecolor = 'b'
        ax.annotate('WBC', xy=(xmax-40,ymin+20))
    elif row.cell_type == 'Platelets':
        edgecolor = 'g'
        ax.annotate('Platelets', xy=(xmax-40,ymin+20))
        
    # add bounding boxes to the image
    rect = patches.Rectangle((xmin,ymin), width, height, edgecolor = edgecolor, facecolor = 'none')
    
    ax.add_patch(rect)

위 그림처럼 결과가 나올수 있도록 학습할 것이다. 이미지에서 다른 클래스끼리 bounding box가 겹쳐있기도 하다.

## Implementing Faster R-CNN

 Faster R-CNN 모델을 [keras-frcnn](https://github.com/kbardool/keras-frcnn.git)을 이용해서 만들 것이다. 해당 저장소를 clone한 후, 내용을 살펴보자.
 
 먼저, `train_frcnn.py`을 실행시키면 주어진 이미지로 학습을 진행하고 model_weights를 pickle로 저장하여 h5 형태로 만들어준다. 학습을 하기위해서는 `filepath,x1,y1,x2,y2,class_name`의 내용이 있는 txt 파일이 필요하므로 위 형식의 파일을 먼저 만들어야 한다.
 
 `test_frcnn.py`을 실행시키면 저장된 weights 값을 불러온 모델에 test image를 통과시켜 결과를 출력한다.

### clone keras-frcnn git repo

In [ ]:
!git clone https://github.com/kbardool/keras-frcnn.git

### Create txt file

In [ ]:
data = pd.DataFrame()
data['format'] = train['prev_filename']

# as the images are in train_images folder, add train_images before the image name
for i in range(data.shape[0]):
    data['format'][i] = 'c:/Users/home/source/BCCD_Dataset/BCCD/JPEGImages/' + data['format'][i]

# add xmin, ymin, xmax, ymax and class as per the format required
for i in range(data.shape[0]):
    data['format'][i] = data['format'][i] + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + train['cell_type'][i]

data.to_csv('./annotate.txt', header=None, index=None, sep=' ')

In [ ]:
data_df = pd.read_csv('./keras-frcnn/annotate.txt', header=None)

In [ ]:
data_df.head()

In [ ]:
c:/Users/home/Research_and_ex/edwith_paperMac/keras-frcnn/model_frcnn.hdf5

In [ ]:
!python ./keras-frcnn/train_frcnn.py  -o simple -p ./annotate.txtb

학습에 시간이 소요되는 관계로 저자가 미리 학습시켜놓은 weights.h5 를 다운받아 test에 사용해보자.

** weights 파일은 코드와 같은 위치에 저장시켜놓으면 수정없이 바로 사용할 수 있다.

In [ ]:
!python ./keras-frcnn/test_frcnn.py -p ./keras-frcnn/test_data/